In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import sys
import re



In [2]:
skillType=pd.read_csv("vizwiz/vizwiz_skill_typ_train.csv",engine="python")

skillType.head(3)

,qid,IMG,QSN,ANS_TYP,TXT,OBJ,COL,CNT,OTH,ANS1,ANS2,ANS3,ANS4,ANS5,ANS6,ANS7,ANS8,ANS9,ANS10
0,VizWiz_train_000000000000.jpg,VizWiz_train_000000000000.jpg,What's the name of this product?,other,1,1,0,0,0,basil leaves,basil leaves,basil,basil,basil leaves,basil leaves,basil leaves,basil leaves,basil leaves,basil
1,VizWiz_train_000000000001.jpg,VizWiz_train_000000000001.jpg,Can you tell me what is in this can please?,other,1,1,0,0,0,soda,coca cola,coca cola,unsuitable,unsuitable,coke 0,coca cola 0,coke 0,coca cola,coke
2,VizWiz_train_000000000002.jpg,VizWiz_train_000000000002.jpg,Is this enchilada sauce or is this tomatoes? ...,other,1,1,0,0,0,these tomatoes not enchilada sauce,tomatoes,tomatoes,tomatoes,tomatoes,crushed tomatoes,crushed tomatoes,tomatoes,tomatoes,tomatoes


In [3]:
keyPhrase=pd.read_csv("vqa/vqa_train_keyphrases.csv", delimiter=";", engine="python",error_bad_lines=False)
keyPhrase.rename(columns={'oth,,,,,':'oth'}, inplace = True)
for i in range(0,len(keyPhrase)):
    keyPhrase.loc[i,'oth']=keyPhrase.loc[i,'oth'].replace(',', '')
keyPhrase.head(3)

Skipping line 47: ';' expected after '"'
Skipping line 108: ';' expected after '"'
Skipping line 186: ';' expected after '"'
Skipping line 211: ';' expected after '"'
Skipping line 238: ';' expected after '"'
Skipping line 309: ';' expected after '"'
Skipping line 338: ';' expected after '"'
Skipping line 479: ';' expected after '"'
Skipping line 551: ';' expected after '"'
Skipping line 569: ';' expected after '"'
Skipping line 813: ';' expected after '"'
Skipping line 856: ';' expected after '"'
Skipping line 1024: ';' expected after '"'
Skipping line 1035: ';' expected after '"'
Skipping line 1153: ';' expected after '"'
Skipping line 1183: ';' expected after '"'
Skipping line 1209: ';' expected after '"'
Skipping line 1223: ';' expected after '"'
Skipping line 1228: ';' expected after '"'
Skipping line 1250: ';' expected after '"'
Skipping line 1279: ';' expected after '"'
Skipping line 1286: ';' expected after '"'
Skipping line 1290: ';' expected after '"'
Skipping line 1370: ';' 

,qid,question,keyPhrases,obj,txt,col,cnt,oth
0,100063007,Are these trains moving?,['trains'],1,0,1,0,1
1,100140002,Are the clouds visible?,['clouds visible'],1,1,0,0,0
2,100142000,What number is on the shorts?,"['number', 'shorts']",1,0,1,1,0


In [8]:
textRecognition1=pd.read_csv("vizwiz/vizwiz_train_text_recognition_1.csv", delimiter=";", engine="python",
                       error_bad_lines=False)
textRecognition2=pd.read_csv("vizwiz/vizwiz_train_text_recognition_2.csv", delimiter=";", engine="python",
                       error_bad_lines=False)
textRecognition3=pd.read_csv("vizwiz/vizwiz_train_text_recognition_3.csv", delimiter=";", engine="python",
                       error_bad_lines=False)
textRecognition=textRecognition1.append(textRecognition2,ignore_index=True)
textRecognition=textRecognition.append(textRecognition3,ignore_index=True)
print(len(textRecognition))
textRecognition.rename(columns={'handwritten_text,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,':'handwritten_text'}, inplace = True)
for i in range(0,len(textRecognition)):
    print(i)
    if textRecognition.loc[i,'handwritten_text']==None:
        textRecognition.loc[i,'handwritten_text']=""
    textRecognition.loc[i,'handwritten_text']=textRecognition.loc[i,'handwritten_text'].replace(',,', '')
    textRecognition.loc[i,'handwritten_text']=textRecognition.loc[i,'handwritten_text'].replace('],', ']')
textRecognition.head(3)

Skipping line 2150: ';' expected after '"'
Skipping line 42: Expected 4 fields in line 42, saw 7
Skipping line 60: Expected 4 fields in line 60, saw 8
Skipping line 157: Expected 4 fields in line 157, saw 5
Skipping line 207: Expected 4 fields in line 207, saw 6
Skipping line 210: Expected 4 fields in line 210, saw 5
Skipping line 275: Expected 4 fields in line 275, saw 5
Skipping line 293: Expected 4 fields in line 293, saw 5
Skipping line 341: Expected 4 fields in line 341, saw 7
Skipping line 357: Expected 4 fields in line 357, saw 5
Skipping line 376: Expected 4 fields in line 376, saw 5
Skipping line 469: Expected 4 fields in line 469, saw 6
Skipping line 478: Expected 4 fields in line 478, saw 5
Skipping line 494: Expected 4 fields in line 494, saw 5
Skipping line 496: Expected 4 fields in line 496, saw 6
Skipping line 504: Expected 4 fields in line 504, saw 5
Skipping line 512: Expected 4 fields in line 512, saw 5
Skipping line 523: Expected 4 fields in line 523, saw 6
Skipping 

7739
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722


5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370


6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003
7004
7005
7006
7007
7008
7009
7010
7011
7012
7013
7014
7015
7016
7017


,qid,question,ocr_text,handwritten_text
0,VizWiz_train_000000000000.jpg,What's the name of this product?,"['b', 'sil', 'leaves', '0.62', 'oz', '(170)']",['NET WT O. 62 02 ( 179)']
1,VizWiz_train_000000000001.jpg,Can you tell me what is in this can please?,[],['^TAKE Three 1^']
2,VizWiz_train_000000000002.jpg,Is this enchilada sauce or is this tomatoes? ...,[],"['ESTI`', 'seamong']"


In [35]:
textRecognition_image=pd.read_csv("vizwiz/api_features_train.csv", engine="python")
textRecognition_image.head(3)

,qid,adult,categories,descriptions,tags
0,VizWiz_train_000000000000.jpg,False,others_,a bottle of wine on a table,'bottle' 'indoor' 'table' 'sitting' 'wine'...
1,VizWiz_train_000000000001.jpg,False,others_,a kitchen with a sink and a microwave,'indoor' 'cabinet' 'kitchen' 'small' 'refr...
2,VizWiz_train_000000000002.jpg,False,others_,a can of soda,'bottle' 'sitting' 'table' 'orange' 'layin...


In [43]:
textResult = pd.merge(textRecognition, skillType, how='left', on='qid')
textResult = pd.merge(textResult,textRecognition_image,  how='left', on='qid')
textResult['answer']=textResult['descriptions']+textResult['tags']

textRecognition.head(3)
textResult

,qid,question,ocr_text,handwritten_text,answer,IMG,QSN,ANS_TYP,TXT,OBJ,...,ANS5,ANS6,ANS7,ANS8,ANS9,ANS10,adult,categories,descriptions,tags
0,VizWiz_train_000000000000.jpg,What's the name of this product?,"['b', 'sil', 'leaves', '0.62', 'oz', '(170)']",['NET WT O. 62 02 ( 179)'],a bottle of wine on a table'bottle' 'indoor' ...,VizWiz_train_000000000000.jpg,What's the name of this product?,other,1.0,1.0,...,basil leaves,basil leaves,basil leaves,basil leaves,basil leaves,basil,False,others_,a bottle of wine on a table,'bottle' 'indoor' 'table' 'sitting' 'wine'...
1,VizWiz_train_000000000001.jpg,Can you tell me what is in this can please?,[],['^TAKE Three 1^'],a kitchen with a sink and a microwave'indoor' ...,VizWiz_train_000000000001.jpg,Can you tell me what is in this can please?,other,1.0,1.0,...,unsuitable,coke 0,coca cola 0,coke 0,coca cola,coke,False,others_,a kitchen with a sink and a microwave,'indoor' 'cabinet' 'kitchen' 'small' 'refr...
2,VizWiz_train_000000000002.jpg,Is this enchilada sauce or is this tomatoes? ...,[],"['ESTI`', 'seamong']",a can of soda'bottle' 'sitting' 'table' 'or...,VizWiz_train_000000000002.jpg,Is this enchilada sauce or is this tomatoes? ...,other,1.0,1.0,...,tomatoes,crushed tomatoes,crushed tomatoes,tomatoes,tomatoes,tomatoes,False,others_,a can of soda,'bottle' 'sitting' 'table' 'orange' 'layin...
3,VizWiz_train_000000000003.jpg,What is the captcha on this screenshot?,"['...1.', 'at&t', 'submit', '4:49', 'pm', 'cli...","['Click here to start download .', '1 3 6 N', ...",a screenshot of a cell phone'screenshot' 'abs...,VizWiz_train_000000000003.jpg,What is the captcha on this screenshot?,other,1.0,1.0,...,t36m,t36m,t36m,t36m,t36m,t63m,False,others_,a screenshot of a cell phone,'screenshot' 'abstract'
4,VizWiz_train_000000000004.jpg,What is this item?,"['bj', 'ofofne', 'powek', 'lfghtfng', 'olar', ...","['OLAR GARDEN/ LIGHT*', 'OLAR GARDEN LIGHT', '...",None'sitting' 'black' 'table' 'green' 'man...,VizWiz_train_000000000004.jpg,What is this item?,other,1.0,1.0,...,solar garden light,garden light,solar garden light,solar garden light,solar garden light,solar garden light,False,others_,None,'sitting' 'black' 'table' 'green' 'man' '...
5,VizWiz_train_000000000006.jpg,What is in this bottle?,"['equgte', 'compare', 'to', 'citrus', 'listeri...","['equate.', 'Compare', 'to Citrus', 'Listerine...",a close up of a bottle'bottle' 'table' 'sitt...,VizWiz_train_000000000006.jpg,What is in this bottle?,other,1.0,1.0,...,advanced antiseptic mouthwash tartar protectio...,antiseptic mouthrinse,mouthwash,mouthwash,mouthwash,mouthrinse,False,others_,a close up of a bottle,'bottle' 'table' 'sitting' 'food' 'cellpho...
6,VizWiz_train_000000000007.jpg,This item.,"['monitor', 'cleaning']","['MONITION', 'CLEANING KIT']",a hand holding a cell phone'person' 'indoor' ...,VizWiz_train_000000000007.jpg,This item.,other,1.0,1.0,...,monitor cleaning kit,monitor cleaner,monitor cleaning kit,monitor cleaning kit,monitor cleaning kit,monitor cleaning kit,False,others_,a hand holding a cell phone,'person' 'indoor' 'hand' 'small' 'sitting'...
7,VizWiz_train_000000000013.jpg,What kind of drink is this?,[],"['HOR', '\\HAVING AGNATHIAANY']",a person lying on a bed'indoor' 'bed' 'layin...,VizWiz_train_000000000013.jpg,What kind of drink is this?,other,1.0,1.0,...,dr pepper,dr pepper,energy drink,dr pepper,dr pepper,dr pepper,False,others_,a person lying on a bed,'indoor' 'bed' 'laying' 'sitting' 'table' ...
8,VizWiz_train_000000000016.jpg,How much money is on the table?,[],[],None'table' 'standing' 'man' 'clock' 'food...,VizWiz_train_000000000016.jpg,How much money is on the table?,other,1.0,1.0,...,80 cents,80 cents,$1,4 coins on table,65 cents,$0.85,False,abstract_nonphoto,None,'table' 'standing' 'man' 'clock' 'food' '...
9,VizWiz_train_000000000020.jpg,What is this game?,[],"['NTSC UIC*', 'F: | |[ !']",a stack of flyers on a table'indoor' 'sitting...,VizWiz_train_000000000020.jpg,What is this game?,other,1.0,1.0,...,grand theft 

In [44]:
#evaluate
n 			  = 2
accuracy     = {}
evalQA       = {}
evalAnsType  = {}
contractions = {"aint": "ain't", "arent": "aren't", "cant": "can't", "couldve": "could've", "couldnt": "couldn't", \
							 "couldn'tve": "couldn't've", "couldnt've": "couldn't've", "didnt": "didn't", "doesnt": "doesn't", "dont": "don't", "hadnt": "hadn't", \
							 "hadnt've": "hadn't've", "hadn'tve": "hadn't've", "hasnt": "hasn't", "havent": "haven't", "hed": "he'd", "hed've": "he'd've", \
							 "he'dve": "he'd've", "hes": "he's", "howd": "how'd", "howll": "how'll", "hows": "how's", "Id've": "I'd've", "I'dve": "I'd've", \
							 "Im": "I'm", "Ive": "I've", "isnt": "isn't", "itd": "it'd", "itd've": "it'd've", "it'dve": "it'd've", "itll": "it'll", "let's": "let's", \
							 "maam": "ma'am", "mightnt": "mightn't", "mightnt've": "mightn't've", "mightn'tve": "mightn't've", "mightve": "might've", \
							 "mustnt": "mustn't", "mustve": "must've", "neednt": "needn't", "notve": "not've", "oclock": "o'clock", "oughtnt": "oughtn't", \
							 "ow's'at": "'ow's'at", "'ows'at": "'ow's'at", "'ow'sat": "'ow's'at", "shant": "shan't", "shed've": "she'd've", "she'dve": "she'd've", \
							 "she's": "she's", "shouldve": "should've", "shouldnt": "shouldn't", "shouldnt've": "shouldn't've", "shouldn'tve": "shouldn't've", \
							 "somebody'd": "somebodyd", "somebodyd've": "somebody'd've", "somebody'dve": "somebody'd've", "somebodyll": "somebody'll", \
							 "somebodys": "somebody's", "someoned": "someone'd", "someoned've": "someone'd've", "someone'dve": "someone'd've", \
							 "someonell": "someone'll", "someones": "someone's", "somethingd": "something'd", "somethingd've": "something'd've", \
							 "something'dve": "something'd've", "somethingll": "something'll", "thats": "that's", "thered": "there'd", "thered've": "there'd've", \
							 "there'dve": "there'd've", "therere": "there're", "theres": "there's", "theyd": "they'd", "theyd've": "they'd've", \
							 "they'dve": "they'd've", "theyll": "they'll", "theyre": "they're", "theyve": "they've", "twas": "'twas", "wasnt": "wasn't", \
							 "wed've": "we'd've", "we'dve": "we'd've", "weve": "we've", "werent": "weren't", "whatll": "what'll", "whatre": "what're", \
							 "whats": "what's", "whatve": "what've", "whens": "when's", "whered": "where'd", "wheres": "where's", "whereve": "where've", \
							 "whod": "who'd", "whod've": "who'd've", "who'dve": "who'd've", "wholl": "who'll", "whos": "who's", "whove": "who've", "whyll": "why'll", \
							 "whyre": "why're", "whys": "why's", "wont": "won't", "wouldve": "would've", "wouldnt": "wouldn't", "wouldnt've": "wouldn't've", \
							 "wouldn'tve": "wouldn't've", "yall": "y'all", "yall'll": "y'all'll", "y'allll": "y'all'll", "yall'd've": "y'all'd've", \
							 "y'alld've": "y'all'd've", "y'all'dve": "y'all'd've", "youd": "you'd", "youd've": "you'd've", "you'dve": "you'd've", \
							 "youll": "you'll", "youre": "you're", "youve": "you've"}
manualMap    = { 'none': '0',
							  'zero': '0',
							  'one': '1',
							  'two': '2',
							  'three': '3',
							  'four': '4',
							  'five': '5',
							  'six': '6',
							  'seven': '7',
							  'eight': '8',
							  'nine': '9',
							  'ten': '10'
							}
articles     = ['a',
							 'an',
							 'the'
							]

 

periodStrip  = re.compile("(?!<=\d)(\.)(?!\d)")
commaStrip   = re.compile("(\d)(\,)(\d)")
punct        = [';', r"/", '[', ']', '"', '{', '}',
				'(', ')', '=', '+', '\\', '_', '-',
				'>', '<', '@', '`', ',', '?', '!']

def processPunctuation(inText):
		outText = inText
		for p in punct:
			if (p + ' ' in inText or ' ' + p in inText) or (re.search(commaStrip, inText) != None):
				outText = outText.replace(p, '')
			else:
				outText = outText.replace(p, ' ')	
		outText = periodStrip.sub("",
									 outText,
									 re.UNICODE)
		return outText
	
def processDigitArticle(inText):
		outText = []
		tempText = inText.lower().split()
		for word in tempText:
			word = manualMap.setdefault(word, word)
			if word not in articles:
				outText.append(word)
			else:
				pass
		for wordId, word in enumerate(outText):
			if word in contractions: 
				outText[wordId] = contractions[word]
		outText = ' '.join(outText)
		return outText

    
def setAccuracy(accQA, accAnsType):
		accuracy['overall']         = round(100*float(sum(accQA))/len(accQA), n)
		accuracy['perAnswerType']   = {ansType:  round(100*float(sum(accAnsType[ansType]))/len(accAnsType[ansType]), n) for ansType in accAnsType}
			
def setEvalQA(quesId, acc):
		evalQA[quesId] = round(100*acc,n)


def setEvalAnsType(quesId, ansType, acc):
		if ansType not in evalAnsType:
			evalAnsType[ansType] = {}
		evalAnsType[ansType][quesId] = round(100*acc, n)



In [45]:
accQA       = []
accAnsType  = {}
for i in range(0,len(textResult)):
    
    resAns      = textResult.loc[i]['answer']
    
    resAns      = resAns.replace('\n', ' ')
    resAns      = resAns.replace('\t', ' ')
    resAns      = resAns.strip()
    resAns      = processPunctuation(resAns)
    resAns      = processDigitArticle(resAns)
    print(resAns)
    gtAcc  = []
    gtAnswers = [str(textResult.loc[i]['ANS1']),
                 str(textResult.loc[i]['ANS2']),
                 str(textResult.loc[i]['ANS3']),
                 str(textResult.loc[i]['ANS4']),
                 str(textResult.loc[i]['ANS5']),
                 str(textResult.loc[i]['ANS6']),
                 str(textResult.loc[i]['ANS7']),
                 str(textResult.loc[i]['ANS8']),
                 str(textResult.loc[i]['ANS9']),
                 str(textResult.loc[i]['ANS10'])]
    print(gtAnswers)
    if len(set(gtAnswers)) > 1: 
        for ansDic in gtAnswers:
            ansDic = processPunctuation(ansDic)
    for gtAnsDatum in gtAnswers:
        otherGTAns = [item for item in gtAnswers if item!=gtAnsDatum]
        matchingAns=[]
        for item in otherGTAns:
            if item.isdigit():
                item=' '+item+' '
            if item=="no" or item=="yes":
                item=' '+item+' '
            if item in resAns:
                matchingAns.append(item)
        acc = min(1, float(len(matchingAns))/3)
        gtAcc.append(acc)

    ansType     = textResult.loc[i]['ANS_TYP']

    avgGTAcc = float(sum(gtAcc))/len(gtAcc)
    print(avgGTAcc)
    accQA.append(avgGTAcc)

    if ansType not in accAnsType:
        accAnsType[ansType] = []

    accAnsType[ansType].append(avgGTAcc)
    setEvalQA(textResult.loc[i]['qid'], avgGTAcc)
    setEvalAnsType(textResult.loc[i]['qid'], ansType, avgGTAcc)


setAccuracy(accQA, accAnsType)
print("Done computing accuracy")
	
	

bottle of wine on table'bottle' 'indoor' 'table' 'sitting' 'wine' 'food' 'glass' 'plate' 'red' 'sandwich' 'laying' 'coffee' 'white'
['basil leaves', 'basil leaves', 'basil', 'basil', 'basil leaves', 'basil leaves', 'basil leaves', 'basil leaves', 'basil leaves', 'basil']
0.0
kitchen with sink and microwave'indoor' 'cabinet' 'kitchen' 'small' 'refrigerator' 'counter' 'sitting' 'white' 'sink' 'microwave' 'wooden' 'brown' 'black' 'steel' 'stove' 'mirror' 'oven' 'holding' 'table' 'stainless' 'man' 'standing' 'room'
['soda', 'coca cola', 'coca cola', 'unsuitable', 'unsuitable', 'coke 0', 'coca cola 0', 'coke 0', 'coca cola', 'coke']
0.0
can of soda'bottle' 'sitting' 'table' 'orange' 'laying' 'cellphone' 'food' 'phone' 'empty' 'can' 'banana' 'black' 'fruit'
['these tomatoes not enchilada sauce', 'tomatoes', 'tomatoes', 'tomatoes', 'tomatoes', 'crushed tomatoes', 'crushed tomatoes', 'tomatoes', 'tomatoes', 'tomatoes']
0.0
screenshot of cell phone'screenshot' 'abstract'
['t36m', 't36m', 't36m'

['green mountain coffee', 'green mountain sumatra reserve', 'extra bold summer reserve', 'green mountain coffee extra bold', 'extra bold sumatran reserve', 'extra bold', 'green mountain', 'green mountain coffee extra bold', 'unanswerable', 'sumatran reserve extra bold']
0.0
empty bottle sitting on table'bottle' 'indoor' 'table' 'sitting' 'laying' 'wine' 'food' 'book' 'banana' 'beer' 'empty' 'lying' 'cheese' 'pizza'
['vitamins', 'melatonin', 'melatonin', 'dietary supplement', 'melatonin', 'melatonin', 'melatonin', 'melatonin', 'melatonin', 'melatonin']
0.0
close up of book'book' 'indoor' 'sitting' 'small' 'open' 'black' 'surface' 'laying' 'counter' 'close' 'empty' 'brush' 'table' 'cat' 'stack' 'bowl' 'food' 'clock' 'phone'
['writing', 'kroger anti diarrheal', 'anti diarrheal medicine', 'anti diarrheal', 'unanswerable', 'krogers anti diarrheal medicine', 'information', 'anti diarrhea', 'anti diarrheal', 'anti diarrheal']
0.0
sandwich sitting on top of wooden table'indoor' 'table' 'sittin

0.0
close up of yellow wall'food' 'yellow' 'hot' 'blurry'
['cereal almonds', 'unsuitable', 'unsuitable', 'cereal', 'almonds', 'unanswerable', 'cereal almonds', 'unanswerable', 'cereal', 'rg']
0.0
person in dark room'sitting' 'dark' 'screen' 'light' 'front' 'laptop' 'monitor' 'computer' 'large' 'standing' 'television' 'man' 'holding' 'white' 'black' 'woman' 'room' 'table' 'clock' 'cat' 'water'
['8', 'unsuitable', '8', '8', 'unanswerable', '8', '8', '8', 'unsuitable', 'unsuitable']
0.0
can of soda'indoor' 'bottle' 'sitting' 'table' 'food' 'counter' 'can' 'filled' 'coffee' 'white' 'group' 'kitchen' 'laying'
['budweiser beer', 'beer', 'beer', 'budweiser', 'budweiser beer', 'beer', 'beer', 'budweiser', 'beer', 'beer can']
0.0
close up of food on plate'indoor' 'food' 'table' 'sitting' 'plate' 'small' 'holding' 'black' 'eating' 'white' 'bowl' 'man' 'hot' 'dog' 'cheese' 'pizza' 'sandwich' 'laying'
['sausage', 'pork sausage', 'pork sausage', 'pork', 'pork sausage', 'unanswerable', 'pork sausage

['white charger rectangle for iphone ipod', 'square white charger', 'charger white rectangular mobile phone', 'charger white rectangle apple product', 'this charger', 'white charger', 'white charger plug', 'off white computer peripheral cable', 'plug', 'white silver ipd charger']
0.0
close up of floor'floor' 'building' 'table'
['no', 'unanswerable', 'no', 'not receipt', 'yes', 'no', 'no buisness card', 'yes no', 'vava', 'no']
0.0
book sitting on top of wooden table'table' 'indoor' 'sitting' 'wooden' 'book' 'laying' 'black' 'food'
['winston', 'winston', 'winston', 'winston', 'winston', 'winston', 'winston', 'winston', 'winston', 'winston']
0.0
stack of flyers on table'indoor' 'sitting' 'table' 'desk' 'laptop' 'white' 'food' 'mirror' 'computer' 'bed' 'laying' 'cat' 'refrigerator'
['dollar', 'money', '1 dollar bill', '1 dollor', '$1 bill', '1 dollar bill', '1 dollar bill', 'dollar bill', '1 dollar bill', 'dollar bill']
0.0
bowl of oranges on table'table' 'food' 'bowl' 'plate' 'sitting' 'w

none'indoor' 'table' 'sitting' 'food' 'sign' 'board' 'wooden' 'bowl' 'orange' 'red' 'holding' 'cake' 'pizza' 'laying' 'display' 'sandwich' 'white' 'plate' 'standing'
['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes']
0.0
bottle next to sink'indoor' 'bottle' 'sitting' 'refrigerator' 'table' 'sink' 'food' 'counter' 'white' 'filled' 'plate' 'kitchen' 'water' 'microwave' 'laying' 'man' 'pizza' 'group'
['triple antibiotic ointment', 'triple antibiotic ointment', 'triple antibiotic ointment', 'tripe antibiotic ointment', 'triple antibiotic ointment plus', 'qdqew', 'triple antibiotic ointment', 'unanswerable', 'triple antibiotic ointment', 'triple antibiotic ointment plus']
0.0
stack of flyers on table'indoor' 'table' 'sitting' 'small' 'desk' 'front' 'wooden' 'wine' 'white' 'computer' 'clock' 'food' 'water' 'kitchen'
['nan', 'pouch', 'nan', 'nan', 'nan', 'nan', 'nan', 'pouch', 'nan', 'pouch']
0.0
none'indoor' 'orange' 'sitting' 'table' 'food' 'toothbrush' 'counter' 'brush'

...

['sun chips', 'french onion', 'french onion', 'sun chips', 'sun french onion chips', 'french onion sun chips', 'french onion', 'french onion sun chips', 'onion', 'french onion']
0.0
book sitting on top of bookshelf'book' 'shelf' 'indoor' 'sitting' 'table' 'computer' 'desk' 'laptop' 'front' 'small' 'room' 'filled' 'office' 'keyboard' 'black' 'living' 'library' 'man' 'bookshelf' 'holding' 'cat' 'remote' 'clock' 'playing' 'video'
['preaching without notes preached without notes', 'book', 'sermons preached without notes', 'unanswerable', 'expository preaching without notes: plus sermons preached without notes', 'unsuitable', 'something preaching without notes', 'preaching without notes', 'unsuitable', 'unanswerable']
0.3
screen shot of computer'indoor' 'black' 'monitor' 'sitting' 'computer' 'table' 'screen' 'white' 'woman' 'store' 'desk' 'standing'
['reboot', 'windows failed to start', 'windows failed to start', 'windows failed to start', 'windows failed to start recent hardware software c

['78 1', '78 1', '78 1', '78 1', '78 1', '78 1', '78 1', '78', '78 1', '78 1']
0.0
none'indoor' 'sitting' 'table' 'white' 'holding' 'black' 'man' 'pizza' 'blue' 'room'
['smoker box', 'smoker box', 'smoker box', 'smoker box', 'smoker box', 'smoker box', 'unanswerable', 'smoker box', 'smoker box', 'cast iron smoker box']
0.0
close up of computer keyboard'electronics' 'computer' 'keyboard' 'indoor' 'sitting' 'black' 'laptop' 'white' 'table' 'mouse' 'desk' 'board' 'close' 'wooden' 'laying' 'man'
['no third key on top', 'yes', 'not 3rd', 'no', 'no', 'no', 'escape', 'yes', 'no', 'no']
0.0
television is sitting on table'indoor' 'table' 'sitting' 'laptop' 'living' 'room' 'computer' 'television' 'black' 'desk' 'glass' 'white' 'rug' 'red' 'phone' 'cat'
['pepsi', 'pepsi', 'someone watching tv', 'finger', 'soda', 'table bottle', 'pepsi bottle on furniture', 'pepsi', 'pepsi', 'pepsi']
0.0
person smiling for camera'person' 'indoor' 'man' 'holding' 'woman' 'looking' 'white' 'smiling' 'wearing' 'young

0.0
bottle next to cup of coffee'indoor' 'sitting' 'food' 'cup' 'table' 'bottle' 'coffee' 'black' 'glass' 'plate' 'phone' 'laying'
['corn', 'corn', 'corn', 'corn', 'corn', 'corn', 'corn', 'corn', 'corn', 'corn']
0.0
sign on screen'monitor' 'screen' 'sign' 'white' 'clock' 'meter' 'red'
['62410 62375', '62410 62375', '62410 62375', '62410 62375', '62410 62375', '62410', '62410 62375', '62410 62375', '62410 62375', 'numbers']
0.0
close up of book'sitting'
['fdgfd', 'chicken casserole', 'chicken casserole', 'chicken casserole', 'unanswerable', 'chicken casserole', 'chicken casserole', 'chicken', 'chicken casserole', 'chicken casserole']
0.0
screen shot of computer'indoor' 'monitor' 'table' 'sitting' 'laptop' 'screen' 'man' 'computer' 'small' 'television' 'black' 'cat' 'room'
['street fighter', 'street fighter', 'street racer', 'street fighter', 'unsuitable', 'street fighter', 'video game', 'cartoons', 'your desktop', 'unsuitable']
0.0
person sitting on table'indoor' 'person' 'table' 'sitti

['cream mushroom', 'campbells', 'campbells', 'creamy', 'campbells', 'mushroom', 'cream mushroom', 'cream mushroom', 'campbells cream mushroom', 'cream mushroom']
0.0
close up of newspaper'text' 'bottle' 'newspaper'
['yes 90% page', 'clear about 80% visible', 'very clear but you cut off right edge', 'some what clear 90 percent', 'medium most', 'mostly clear about 3 4ths page visible', 'clear almost all page', 'very clear', 'all text clear in full page', 'can almost read all words bit blurry but big enough']
0.0
blurry image of person'photo' 'looking' 'light' 'sitting' 'dark' 'stop' 'traffic' 'black' 'cat' 'red' 'table' 'man' 'holding' 'standing' 'sign' 'cake' 'room' 'white' 'street' 'blurry' 'phone'
['music', 'picture', 'unanswerable', 'person', 'unanswerable', 'ear training', 'ear training ad', 'book', 'album', 'ear training']
0.3
none'indoor' 'toothbrush' 'sitting' 'brush' 'teeth' 'food' 'close' 'holding' 'man' 'young' 'table' 'refrigerator' 'brushing' 'laying' 'sink' 'kitchen' 'bed' 

AttributeError: 'float' object has no attribute 'replace'

In [46]:
setAccuracy(accQA, accAnsType)
print("Done computing accuracy")

Done computing accuracy


In [47]:
print(accQA)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.3, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.3, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5333333333333333, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.0, 0.0, 0.0, 0.0, 0.3, 0.5333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [48]:
print(accAnsType)

{'number': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'unanswerable': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [49]:
print(accuracy['overall'])         
print(accuracy['perAnswerType'])  

4.74
{'number': 0.26, 'other': 5.31, 'unanswerable': 3.45, 'yes/no': 0.77}
